In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1646191796205_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [2]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# sc.install_pypi_package("Cython")
# sc.install_pypi_package("pandas")
# sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## File Selection Criteria

Data from 2019 to 2021 - Timestamp starts with 1 and ends with either 2 or 7.

In [5]:
# f = !aws s3 ls --human-readable --recursive \
# s3://openaq-fetches/realtime/ | grep '20[12][901]-[0-9][0-9]-[0-3][0-9]' | grep '1*5.ndjson'
    
# size = sum(list(map(lambda i : float(i.split()[2]) if i.split()[3] == 
#                     'MiB' else float(i.split()[2])/1000,f))) /1000

# print(f'Raw Data Size {size :.2f} GB' )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# %%local
# %matplotlib inline

# # plt.plot([1,2,3,4])
# # plt.show()
# import pandas as pd
# import matplotlib
# import matplotlib.pyplot as plt
# import boto3

# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None) 
# pd.set_option('display.max_rows',None)


# matplotlib.style.use('ggplot')
# plt.rcParams["figure.figsize"] = (10, 7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Load Data to Spark

In [2]:
aq = spark.read.json(
    's3://openaq-fetches/realtime/20[12][901]-[0-9][0-9]-[0-3][0-9]'
    '/*.ndjson')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
aq.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _corrupt_record: string (nullable = true)
 |-- attribution: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- averagingPeriod: struct (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- unitit: string (nullable = true)
 |    |-- value: double (nullable = true)
 |-- city: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- local: string (nullable = true)
 |    |-- utc: string (nullable = true)
 |-- location: string (nullable = true)
 |-- mobile: boolean (nullable = true)
 |-- parameter: string (nullable = true)
 |-- sourType: string (nullable = true)
 |-- sourceName: string (nullable = true)
 |-- sourceType: string (nullable = true)
 |-- unit: string (null

In [4]:
aq.createOrReplaceTempView('aq')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Setup temporary tables

List all Countries, Cities  and their coordinates

In [ ]:
q = """
create table distinct_cities as
select distinct country,
    city, 
    coordinates.latitude as lat,
    coordinates.longitude as long
from aq 
"""
spark.sql(q)

In [ ]:
spark.sql('select * from distinct_cities limit 10').show()

List of all parameters

In [ ]:
q = """
create table distinct_params as
select distinct parameter
from aq 
"""
spark.sql(q)

In [ ]:
spark.sql('select * from distinct_params limit 10').show()

In [12]:
q = """
select * 
from (
select * from aq 
where NOT (parameter = 'pm25' and (value < 0 or value >  1000))
) a
where parameter = 'pm25'
limit 100
"""
spark.sql(q).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+------+---------+--------+--------------------+----------+-----+-----+
|_corrupt_record|         attribution|    averagingPeriod|                city|         coordinates|country|                date|            location|mobile|parameter|sourType|          sourceName|sourceType| unit|value|
+---------------+--------------------+-------------------+--------------------+--------------------+-------+--------------------+--------------------+------+---------+--------+--------------------+----------+-----+-----+
|           null|[{StateAir.net, h...| {hours, null, 1.0}|            Shanghai|     {31.21, 121.44}|     CN|{2019-01-28T11:00...|            Shanghai| false|     pm25|    null|            Shanghai|government|µg/m³| 51.0|
|           null|[{StateAir.net, h...| {hours, null, 1.0}|            Shenyang|     {41.78, 123.42}|     CN|{2019-01

In [15]:
spark.sql("drop table aq_per_day")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [ ]:
q = """
create table aq_per_day as
select to_date(to_timestamp(date.utc)) as date, 
            country,
            city, 
            parameter, 
            unit as orig_unit,
            'µg/m³' as unit_new,
            mean(case when unit = 'ppm' then value * 1000 else value end) as mean_value,
            percentile_approx(case when unit = 'ppm' then value * 1000 else value end, .5) as median_value,
            min(case when unit = 'ppm' then value * 1000 else value end) as min_value,
            max(case when unit = 'ppm' then value * 1000 else value end) as max_value,
            stddev(case when unit = 'ppm' then value * 1000 else value end) as std_value
from (
        select * from aq 
        where NOT (parameter = 'pm25' and (value < 0 or value >  1000))        
    ) as s
where to_date(to_timestamp(date.utc)) is not null 
group by 1,2,3,4,5
having mean(case when unit = 'ppm' then value * 1000 else value end) is not null
"""
spark.sql(q)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# spark.sql('select * from aq_per_day limit 10').show()
# aq = spark.read.json('s3://bdcc2022/aq_per_day/*.json')
# aq.createOrReplaceTempView('aq')

spark.sql("select * from aq_per_day where parameter = 'pm25' limit 20").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+--------------------+---------+---------+--------+--------------------+------------+---------+---------+-------------------+
|      date|country|                city|parameter|orig_unit|unit_new|          mean_value|median_value|min_value|max_value|          std_value|
+----------+-------+--------------------+---------+---------+--------+--------------------+------------+---------+---------+-------------------+
|2019-01-06|     US|Youngstown-Warren...|     pm25|    µg/m³|   ?g/m?|                -0.7|        -0.8|     -0.8|     -0.6|0.14142135623730956|
|2019-01-13|     US|        Fayetteville|     pm25|    µg/m³|   ?g/m?|  -2.076923076923077|        -2.0|     -4.0|     -1.0| 1.1875421719907089|
|2019-01-15|     US|Grand Rapids-Wyoming|     pm25|    µg/m³|   ?g/m?|                -1.8|        -1.8|     -1.8|     -1.8|               null|
|2019-01-20|     IN|              Mumbai|     pm25|    µg/m³|   ?g/m?|              -999.0|      -999.0|   -999.0|   -999.0|      

In [9]:
spark.sql('select count(*) from aq_per_day').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
| 8304813|
+--------+

To save processing time and EMR resources, tables are exported to csv in s3 and processing will be done in jojie

In [10]:
(
spark.sql('select * from aq_per_day').write.format("json").mode("Overwrite")
.save("s3a://bdcc2022/aq_per_day")
)

(
spark.sql('select * from distinct_params').write.format("json").option("header","true").mode("Overwrite")
.save("s3a://bdcc2022/distinct_params")
)

(
spark.sql('select * from distinct_cities').write.format("json").option("header","true").mode("Overwrite")
.save("s3a://bdcc2022/distinct_cities")
)   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…